<a href="https://colab.research.google.com/github/ibenoam/Neural_cryptography/blob/main/Neural_Cryptography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural-Cryptography:
### Solving the key exchange problem by synchronize Tree Parity Machines

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
N = 1000
L = 3 
K = 3

num_sys = 100
step_converage = np.zeros((1,num_sys))
sync_time = np.zeros((1,num_sys))
attacker_time = np.zeros((1,num_sys))

In [ ]:
class TPM:
  def __init__(self,N,K,L,M=1):
    # Constants
    self.N = N
    self.K = K
    self.L = L
    self.M = M

    # Initialization
    self.w = np.floor((2*L+1) * np.random.rand(N,K)) - L
    self.fields = np.zeros((1,K))
    self.sigmas = np.zeros((1,K))
    self.tau = 0

  def forward(self,input):
    self.fields = np.sum(self.w*input, axis=0)
    self.sigmas = np.sign(self.fields)
    self.tau = np.prod(self.sigmas)  
  
  def update(self,input):
    update_indexes = np.where(self.sigmas == self.tau)
    self.w[:,update_indexes] += input[:,update_indexes]
    self.w[np.abs(self.w)>self.L] = np.sign(self.w[np.abs(self.w)>self.L])*self.L

In [ ]:
inputs = np.sign(np.random.rand(N,K)-0.5)

alice = TPM(N,K,L)
bob = TPM(N,K,L)


alice.forward(inputs)
bob.forward(inputs)

when_move = np.array([])
count_steps = 0
while not np.array_equal(alice.w,bob.w):
  # print("After {} steps.".format(count_steps))
  count_steps += 1
  inputs = np.sign(np.random.rand(N,K)-0.5)
  alice.forward(inputs)
  bob.forward(inputs)
  
  if alice.tau == bob.tau:
    when_move=np.append(when_move,1)
    alice.update(inputs)
    bob.update(inputs)
  else:
    when_move=np.append(when_move,0)
  

In [ ]:
def entropy_calc(arr):
  